# Implementação DQN com prioritized replay no ambiente Swift com manipulador panda.

Replay buffer <br>
DNN para representar o agente atual e uma DNN alvo com uma taxa de atualização menor.

In [124]:
import torch
import json
import pickle
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output
from scipy.signal import convolve, gaussian
import os
import io
import base64
import time
import glob
from IPython.display import HTML

import data_panda as rbt
device = 'cpu'
%matplotlib inline


#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76

import torch
import json
import pickle
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output
from scipy.signal import convolve, gaussian
import os
import io
import base64
import time
import glob
from IPython.display import HTML

import data_panda as rbt
device = 'cpu'
%matplotlib inline


#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76

### Sistema observável e com medidas dos ângulos disponíveis


    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Joint1                   -4.8                    4.8
        1       Joint2                    -Inf                    Inf
        2       Joint3                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        
    Actions:
        Type: Discrete(9)
        Num   Three actions for each joint
        0     decrement joint j
        1     increment joint j
        2     decrement join  j

        #j3 range 0.0 a 3.7
        #j2 range 0.0 a -3.
        #j1 range -1.7 a 1.7


        



In [125]:
state_shape = 3
env=rbt.Panda_RL()
config = rbt.load_config("config_dueling.yaml")
agent=rbt.DuelingDQNAgent(state_shape, device,layers=config["dueling_layers"], epsilon=0)
env.renderize=True #stop robot viewing

In [126]:
env.panda

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

### Main loop



Intializing with a given state

In [127]:
env.start_scene()

In [128]:
q_far=np.array([ 0., -0.8 ,  0. , -0.0698,  0.,  3.3825,  0.    ])
print(env.reach_joint_limit())

[False, True, False]


In [129]:
rbt.evaluate(env, agent, n_games=1, greedy=False, t_max=100)

/home/taniel/mambaforge/envs/RTB/lib/python3.10/site-packages/roboticstoolbox/robot/Link.py:1041: FutureWarning: base kwarg is deprecated, use pose instead
  warn("base kwarg is deprecated, use pose instead", FutureWarning)


(-10.0, 0.0, 1.0, 0.0, 4.325058789904212, [['Termination', 'Collided']])

In [140]:
# env.reset_j1=[-1.7,1.7]
# env.reset_j1=[-1.7,-1.]

# env.reset_j1= [-0.3,0.3] #[-1.7,-1.] 
# env.reset_j2= [-1,0.06]
# env.reset_j3= [0,3.75]
env.reset()

array([-1.41, -2.67,  0.95])

In [8]:
a=np.array([-1,0,1])

env.step(a)

(array([-1.51, -1.3 ,  1.27]), -20.69015484413311, False, ['Running', ''])

In [9]:
env.panda.q=env.q_goal
env.sig_R=1
print(env.step([1,1,-1]))
print(env.fitness())
# print(env.step([0,0,1]))
# print(env.fitness())

(array([ 0.36, -0.83,  3.68]), 10.705349723499719, False, ['Running', ''])
(3.2599151422785155, 0.010000000000023646)


In [10]:
print(env.step([0,0,-1]))
print(env.fitness())

(array([ 0.36, -0.83,  3.67]), -19.978592555257954, False, ['Running', ''])
(3.258628002016973, 0.020000000000012206)


In [11]:
print(env.step([1,0,0]))
print(env.fitness())

(array([ 0.37, -0.83,  3.67]), 9.57211836358707, False, ['Running', ''])
(3.243047313646215, 0.029999999999999107)


In [12]:

print(env.step([0,1,-1]))

#print(env.fitness())
#j=[(-1.8,1.76), (-3,0),(0,3.75)]




(array([ 0.37, -0.82,  3.66]), -20.017643907299707, False, ['Running', ''])


In [14]:
env.get_q()

[1.4190038033490282e-15, -0.7299999999999993, 1.5000000000000062]

In [141]:
folder="new_19"
#env.reset_j1=[-1.7,-1.]
agent.play(env,folder,tmax=500)

Final score:226.6956649587397 in 500 steps, minimum distance 2.566571467621049
Status: Running 


In [142]:
agent.play(env,folder,tmax=500,model="other")


Final score:-1022.0979154542931 in 500 steps, minimum distance 2.664438439458465
Status: Running 


In [32]:
agent.play(env,folder,tmax=500,model="last")

Final score:199.52234279844146 in 431 steps, minimum distance 2.509509150684599
Status: Termination Collided


In [152]:
DIR='runs/results_25/'
print(glob.glob(DIR+'/*.pt'))

['runs/results_25/best-model-loss.pt', 'runs/results_25/best-model-rw.pt']


**Let us record a video of trained agent**

In [72]:
# # Animate learned policy
# save_dir='./videos/'
# #env = make_env(env_name)
# generate_animation(env, agent, save_dir=save_dir)
# [filepath] = glob.glob(os.path.join(save_dir, '*.mp4'))

# display_animation(filepath)

In [73]:
#TODO

# Run this on another environment in OpenAI Gym
# Create a robotic environment with more actions
#